In [ ]:
!pip install librosa

In [1]:
# 音源の読み込み
import numpy as np
import matplotlib.pyplot as plt
import librosa
import glob
import json
%matplotlib inline

# # 音源の読み込み
# fname = "../Q00001_ja.wav"
# sound = librosa.load(fname, sr=44100, mono=True)  # mono
# # print(tyoe(sound))

# wav_Origin = sound[0]
# size = len(wav_Origin)
# fr = sound[1]
# print(fr, len(wav_Origin) / fr)

# # グラフ化
# t = np.linspace(0, size / fr, len(wav_Origin))
# plt.plot(t, wav_Origin, c="red")
# plt.show()

In [ ]:
# 再生
from IPython.display import Audio
Audio(fname, rate=fr)

In [ ]:
# # 音源の読み込み
# import numpy as np
# import matplotlib.pyplot as plt
# import librosa
# %matplotlib inline

# # 音源の読み込み
# fname2 = "./JKspeech/J01.wav"  # 片側が強い
# sound = librosa.load(fname2, sr=44100, mono=True)  # mono
# wav_Target = sound[0]
# size = len(wav_Target)
# fr = sound[1]
# print(fr, len(wav_Target), len(wav_Target) / fr)

# # グラフ化
# t = np.linspace(0, size / fr, len(wav_Target))
# plt.plot(t, wav_Target, c="blue")
# plt.show()

In [ ]:
# # option
# # 再生
# from IPython.display import Audio
# Audio(fname2, rate=fr)

In [ ]:
# for i in range(1, 45):
#     fname2 = f"./JKspeech/J{i:02d}.wav"  # 片側が強い
#     sound = librosa.load(fname2, sr=44100, mono=True)  # mono
#     wav_Target = sound[0]
    
#     # サイズ調整
#     min_ = np.min([len(wav_Origin), len(wav_Target)])
#     wav_Target = wav_Target[:min_]
#     wav_Origin = wav_Origin[:min_]
    
#     # フーリエ変換
#     fft_O = np.fft.fft(wav_Origin)
#     fft_T = np.fft.fft(wav_Target)

#     # 畳み込み処理の本体
#     fft_Oc = np.conj(fft_O)     # 複素共役を取る
#     sig_c = fft_Oc * fft_T      # 要素ごとに乗算
#     sig_t = np.fft.ifft(sig_c)  # 逆フーリエ変換
#     sig_t = np.abs(sig_t)       # そのままでは複素数なので、スカラー値にする
    
#     print(i, max(sig_t))

In [ ]:
# def fft_targets(wav_Targets):
#     targets = []
#     for wav_target in wav_Targets:
#         fft_T = np.fft.fft(wav_target)
#         targets.append(fft_T)

#     return targets

In [17]:
audio_dir = '../JKspeech/'
fnames = glob.glob(f'{audio_dir}/J*.wav')
sounds = []
for fname in fnames:
    sounds.append(librosa.load(fname, sr=48000, mono=True)[0])
    
print("Done")

Done


In [18]:
def audio_matching(wav_Origin_init, wav_Targets_init):
    result = []
    for i, wav_Target_init in enumerate(wav_Targets_init):
        # サイズ調整
        min_ = np.min([len(wav_Origin_init), len(wav_Target_init)])
        wav_Target = wav_Target_init[:min_]
        wav_Origin = wav_Origin_init[:min_]
    
        # フーリエ変換
        fft_O = np.fft.fft(wav_Origin)
        fft_T = np.fft.fft(wav_Target)
        
        # 畳み込み処理の本体
        fft_Oc = np.conj(fft_O)     # 複素共役を取る
        sig_c = fft_Oc * fft_T     # 要素ごとに乗算
        sig_t = np.fft.ifft(sig_c)  # 逆フーリエ変換
        sig_t = np.abs(sig_t)       # そのままでは複素数なので、スカラー値にする
        result.append(np.max(sig_t))
        
    return np.array(result)

# def eval_result(or_lab, predict):
    

In [19]:
q_id = "Q050_ja"
fname = f"../questions_ja/{q_id}.wav"
sound = librosa.load(fname, sr=48000, mono=True)[0]  # mono

# size = len(sound)
# fr = sound[1]
# print(fr, len(sound) / fr)
# t = np.linspace(0, size / fr, len(sound))
# plt.plot(t, sound, c="red")
# plt.show()

result = audio_matching(sound, sounds)
print("Done")

Done


In [59]:
f = open('../questions_ja/metadata.json','r',encoding="utf-8_sig")
data = json.load(f)
# print(len(data))
metadata = data[q_id]
print(metadata)
f.close()

{'length': 6, 'correct_numbers': [14, 15, 21, 26, 29, 42]}


In [60]:
answer = []
for i in np.argsort(result)[::-1][:metadata['length']]:
    print(i+1, result[i])
    answer.append(i+1)

correct_numbers = metadata['correct_numbers']
correct_numbers_set = set(correct_numbers)
matches = sorted(set(answer) & set(correct_numbers_set))
shortages = [n for n in correct_numbers if n not in answer]

print("\n問題ナンバー:", q_id)
print("正答:", correct_numbers)
print("回答:", sorted(answer))
print("一致:", matches)
print("不足:", shortages)
print(f"正答率: {(len(matches)/metadata['length'])*100:.02f}%")

14 241.08933469466865
8 197.31909599434584
21 183.3268370367587
30 152.9635336017236
28 151.863135850057
22 151.0702113090083

問題ナンバー: Q050_ja
正答: [14, 15, 21, 26, 29, 42]
回答: [8, 14, 21, 22, 28, 30]
一致: [14, 21]
不足: [15, 26, 29, 42]
正答率: 33.33%
